<a href="https://colab.research.google.com/github/DRose1991/Viscous-Shear-Cosmology-Simulation/blob/main/VSC_Model_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install emcee corner

import numpy as np
from scipy.optimize import minimize
import emcee

# 1. DATA
cc_data = np.array([
    [0.070, 69.0, 19.6], [0.090, 69.0, 12.0], [0.120, 68.6, 26.2],
    [0.170, 83.0, 8.0],  [0.179, 75.0, 4.0],  [0.199, 75.0, 5.0],
    [0.200, 72.9, 29.6], [0.270, 77.0, 14.0], [0.280, 88.8, 36.6],
    [0.350, 82.7, 8.4],  [0.352, 83.0, 14.0], [0.380, 81.5, 1.9],
    [0.400, 95.0, 17.0], [0.400, 82.0, 5.6],  [0.425, 87.1, 11.2],
    [0.440, 82.6, 7.8],  [0.470, 89.0, 50.0], [0.478, 80.9, 9.0],
    [0.480, 97.0, 62.0], [0.593, 104.0, 13.0], [0.680, 92.0, 8.0],
    [0.781, 105.0, 12.0],[0.875, 125.0, 17.0], [0.880, 90.0, 40.0],
    [0.900, 117.0, 23.0],[1.037, 154.0, 20.0], [1.300, 168.0, 17.0],
    [1.363, 160.0, 33.6],[1.430, 177.0, 18.0], [1.530, 140.0, 14.0],
    [1.750, 202.0, 40.0],[1.965, 186.5, 50.4]
])

z_obs = cc_data[:, 0]
Hz_obs = cc_data[:, 1]
Hz_err = cc_data[:, 2]

# 2. MODELS

def Hz_VSC(z, H0, Om, O_visc):
    # VSC Model: H(z) = H0 * sqrt(Om(1+z)^3 + O_visc)
    # 3 Parameters: H0, Om, O_visc
    E_sq = Om * (1 + z)**3 + O_visc
    return H0 * np.sqrt(np.abs(E_sq))

def Hz_LCDM(z, H0, Om):
    # Flat LCDM Model: H(z) = H0 * sqrt(Om(1+z)^3 + (1-Om))
    # 2 Parameters: H0, Om
    E_sq = Om * (1 + z)**3 + (1.0 - Om)
    return H0 * np.sqrt(np.abs(E_sq))

# 3. LIKELIHOODS (Negative Log Likelihood for Minimization)

def neg_log_likelihood_VSC(theta, z, y, yerr):
    H0, Om, O_visc = theta
    model = Hz_VSC(z, H0, Om, O_visc)
    sigma2 = yerr**2
    return 0.5 * np.sum((y - model)**2 / sigma2 + np.log(2 * np.pi * sigma2))

def neg_log_likelihood_LCDM(theta, z, y, yerr):
    H0, Om = theta
    model = Hz_LCDM(z, H0, Om)
    sigma2 = yerr**2
    return 0.5 * np.sum((y - model)**2 / sigma2 + np.log(2 * np.pi * sigma2))

# 4. OPTIMIZATION (Finding Max Likelihood)

# Initial Guesses
initial_VSC = [67.0, 0.3, 0.7]
initial_LCDM = [67.0, 0.3]

# Minimize NLL
sol_VSC = minimize(neg_log_likelihood_VSC, initial_VSC, args=(z_obs, Hz_obs, Hz_err), bounds=((50, 100), (0, 1), (0, 1.5)))
sol_LCDM = minimize(neg_log_likelihood_LCDM, initial_LCDM, args=(z_obs, Hz_obs, Hz_err), bounds=((50, 100), (0, 1)))

# 5. AIC / BIC CALCULATION
# AIC = 2k - 2ln(L) = 2k + 2 * NLL_min
# BIC = k*ln(n) - 2ln(L) = k*ln(n) + 2 * NLL_min

n = len(z_obs)

# VSC
k_VSC = 3
nll_VSC = sol_VSC.fun
aic_VSC = 2 * k_VSC + 2 * nll_VSC
bic_VSC = k_VSC * np.log(n) + 2 * nll_VSC

# LCDM
k_LCDM = 2
nll_LCDM = sol_LCDM.fun
aic_LCDM = 2 * k_LCDM + 2 * nll_LCDM
bic_LCDM = k_LCDM * np.log(n) + 2 * nll_LCDM

# Evidence Ratios (relative to best model)
min_aic = min(aic_VSC, aic_LCDM)
delta_aic_VSC = aic_VSC - min_aic
delta_aic_LCDM = aic_LCDM - min_aic

min_bic = min(bic_VSC, bic_LCDM)
delta_bic_VSC = bic_VSC - min_bic
delta_bic_LCDM = bic_LCDM - min_bic

print(f"--- QUANTITATIVE MODEL COMPARISON ---")
print(f"Data Points (n): {n}")
print(f"\nModel: Viscous Shear Cosmology (VSC)")
print(f"Parameters (k): {k_VSC} (H0, Om, O_visc)")
print(f"Best Fit: H0={sol_VSC.x[0]:.2f}, Om={sol_VSC.x[1]:.2f}, O_visc={sol_VSC.x[2]:.2f}")
print(f"AIC: {aic_VSC:.2f} (Delta: {delta_aic_VSC:.2f})")
print(f"BIC: {bic_VSC:.2f} (Delta: {delta_bic_VSC:.2f})")

print(f"\nModel: Standard Flat LCDM")
print(f"Parameters (k): {k_LCDM} (H0, Om)")
print(f"Best Fit: H0={sol_LCDM.x[0]:.2f}, Om={sol_LCDM.x[1]:.2f}")
print(f"AIC: {aic_LCDM:.2f} (Delta: {delta_aic_LCDM:.2f})")
print(f"BIC: {bic_LCDM:.2f} (Delta: {delta_bic_LCDM:.2f})")

print(f"\n--- INTERPRETATION ---")
if delta_aic_VSC < 2 and delta_aic_LCDM < 2:
    print("Both models are statistically indistinguishable (Delta AIC < 2).")
elif aic_VSC < aic_LCDM:
    print("VSC is preferred by AIC.")
else:
    print("Flat LCDM is preferred by AIC (Occam's Razor: fewer parameters).")

--- QUANTITATIVE MODEL COMPARISON ---
Data Points (n): 32

Model: Viscous Shear Cosmology (VSC)
Parameters (k): 3 (H0, Om, O_visc)
Best Fit: H0=66.99, Om=0.33, O_visc=0.66
AIC: 254.48 (Delta: 2.00)
BIC: 258.88 (Delta: 3.47)

Model: Standard Flat LCDM
Parameters (k): 2 (H0, Om)
Best Fit: H0=66.60, Om=0.33
AIC: 252.48 (Delta: 0.00)
BIC: 255.41 (Delta: 0.00)

--- INTERPRETATION ---
Both models are statistically indistinguishable (Delta AIC < 2).
